### 2. Digital Analysis
---
    1. How many users are there?

In [1]:
SELECT 
    COUNT(DISTINCT user_id) as num_users
FROM users

(1 row affected)

Total execution time: 00:00:00.007

num_users
500


    2. How many cookies does each user have on average?

In [2]:
SELECT
    ROUND(CAST(COUNT(cookie_id)as float)/COUNT(DISTINCT user_id),2) AS avg_num_cookies_per_user
FROM users

(1 row affected)

Total execution time: 00:00:00.016

avg_num_cookies_per_user
3.56


    3. What is the unique number of visits by all users per month?

In [3]:
SELECT
    MONTH(event_time) as month,
    COUNT(DISTINCT visit_id) as num_visit
FROM events
GROUP BY MONTH(event_time)
ORDER BY MONTH(event_time)

(5 rows affected)

Total execution time: 00:00:00.101

month,num_visit
1,876
2,1488
3,916
4,248
5,36


    4. What is the number of events for each event type?

In [4]:
SELECT 
    e.event_type,
    event_name,
    COUNT(*)as num_events
FROM events e
JOIN event_identifier ei ON ei.event_type = e.event_type 
GROUP BY e.event_type, event_name
ORDER BY e.event_type

(5 rows affected)

Total execution time: 00:00:00.037

event_type,event_name,num_events
1,Page View,20928
2,Add to Cart,8451
3,Purchase,1777
4,Ad Impression,876
5,Ad Click,702


    5. What is the percentage of visits which have a purchase event?

In [5]:
SELECT 
    event_name,
    COUNT(DISTINCT visit_id) AS num_visit_with_purchase,
    ROUND(CAST(COUNT(DISTINCT visit_id) as float)*100/(SELECT COUNT(DISTINCT visit_id) FROM events),2) as perc_visit_with_purchase
FROM events e 
JOIN event_identifier ei ON ei.event_type = e.event_type
WHERE e.event_type = 3
GROUP BY event_name

(1 row affected)

Total execution time: 00:00:00.138

event_name,num_visit_with_purchase,perc_visit_with_purchase
Purchase,1777,49.86


    6. What is the percentage of visits which view the checkout page but do not have a purchase event?

In [6]:
SELECT 
    100*(1 - CAST((SELECT COUNT(DISTINCT visit_id) FROM events WHERE event_type = 3) as float)/COUNT(DISTINCT visit_id)) 
    as percent_not_purchase
FROM events 
WHERE page_id = 12 

(1 row affected)

Total execution time: 00:00:00.048

percent_not_purchase
15.501664289110796


    7. What are the top 3 pages by number of views?

In [7]:
SELECT TOP 3
    page_name,
    COUNT(*) AS num_views
FROM events e
JOIN page_hierarchy p ON p.page_id = e.page_id 
WHERE event_type = 1
GROUP BY page_name
ORDER BY num_views DESC


(3 rows affected)

Total execution time: 00:00:00.133

page_name,num_views
All Products,3174
Checkout,2103
Home Page,1782


    8. What is the number of views and cart adds for each product category?

In [8]:
SELECT
    product_category,
    SUM(CASE WHEN e.event_type = 1 THEN 1 END) AS num_views,
    SUM(CASE WHEN e.event_type = 2 THEN 1 END) AS num_cart_adds
FROM events e 
JOIN page_hierarchy p ON p.page_id = e.page_id
WHERE product_category is not NULL
GROUP BY product_category

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.061

product_category,num_views,num_cart_adds
Fish,4633,2789
Luxury,3032,1870
Shellfish,6204,3792


    9. What are the top 3 products by purchases?

In [9]:
WITH tbl AS(
    SELECT visit_id
    FROM events
    WHERE event_type = 3
)
SELECT TOP 3
    page_name,
    COUNT(*) as num_purchases
FROM events e
JOIN tbl ON tbl.visit_id = e.visit_id
JOIN page_hierarchy p ON p.page_id = e.page_id
WHERE event_type = 2
GROUP BY page_name
ORDER BY num_purchases DESC

(3 rows affected)

Total execution time: 00:00:00.051

page_name,num_purchases
Lobster,754
Oyster,726
Crab,719
